In [1]:
import os

In [2]:
%pwd

'/Users/khushi/Desktop/Projects/Text-Summarizer/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/khushi/Desktop/Projects/Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [8]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

/Users/khushi/Desktop/Projects/Text-Summarizer/text/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import os
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Trainer,
    TrainingArguments
)
from datasets import load_from_disk


class ModelTrainer:
    def __init__(self, config):
        self.config = config

    def train(self):
        # 🚨 HARD DISABLE MPS
        os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
        os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"

        torch.backends.mps.is_available = lambda: False
        torch.backends.mps.is_built = lambda: False

        device = torch.device("cpu")

        print("✅ Training on:", device)

        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)

        model = AutoModelForSeq2SeqLM.from_pretrained(
            self.config.model_ckpt
        ).to(device)

        data_collator = DataCollatorForSeq2Seq(
            tokenizer=tokenizer,
            model=model
        )

        dataset = load_from_disk(self.config.data_path)

        # 🔽 EXTREMELY IMPORTANT FOR MAC
        dataset["train"] = dataset["train"].select(range(30))
        dataset["validation"] = dataset["validation"].select(range(10))

        training_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=1,

            per_device_train_batch_size=1,
            per_device_eval_batch_size=1,

            gradient_accumulation_steps=1,
            save_strategy="no",

            fp16=False,
            bf16=False,

            logging_steps=20,
            report_to="none",
            no_cuda=True   # 🚨 THIS IS CRITICAL
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            tokenizer=tokenizer,
            data_collator=data_collator,
            train_dataset=dataset["train"],
            eval_dataset=dataset["validation"]
        )

        trainer.train()

        model.save_pretrained(
            os.path.join(self.config.root_dir, "pegasus-samsum-model")
        )
        tokenizer.save_pretrained(
            os.path.join(self.config.root_dir, "tokenizer")
        )


In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2026-01-04 19:58:45,786: INFO: common: yaml file: config/config.yaml loaded successfully]
[2026-01-04 19:58:45,788: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-04 19:58:45,788: INFO: common: created directory at: artifacts]
[2026-01-04 19:58:45,788: INFO: common: created directory at: artifacts/model_trainer]
✅ Training on: cpu


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Python(72656) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
/Users/khushi/Desktop/Projects/Text-Summarizer/text/lib/python3.10/site-packages/transformers/training_args.py:1636: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
/var/folders/pp/sl_6jwq952v8wx21lkz3cybm0000gn/T/ipykernel_72630/3068166633.py:64: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the m

Step,Training Loss
20,2.795000


/Users/khushi/Desktop/Projects/Text-Summarizer/text/lib/python3.10/site-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
